# The IMAGE_META module Elevator Pitch

**_I am a photographer who is tired of maintaining image metadata (EXIF, ITPC, ...) in various tools, especially normal tags and geo coordinates and geo reverse tags like location, city, ... and so on._**

**_This Python module provides a command line level solution to this problem by using exiftool to write data into jpg images._**

# Showcasing IMAGE_META package
The GPS_WRITER_SHOWCASE notbook provides numerous manipulation features for manipulating jpg Image metadata leveraging the great [EXIF Tool](https://exiftool.org/) and using open street map geo API https://nominatim.org/release-docs/develop/api/Overview/ for getting geo meta data.

To get more information on IPTC-IIM metadata (International Press Telecommunications Council-Information Interchange Model) check out the following documentation sources: https://www.iptc.org/std/photometadata/documentation/

The package contains the following modules:

* **geo.py** coordinate calculations, access to nominatim API for reverse geo encoding (coordinates to site plain text information), gpx file handling
* **persistence.py** reading + writing plain + json files
* **exif.py** exiftool interface + image metadata handling / transformation 
* **util** datetime calculations, binary search in list, ...

Caveat: Mind the usage terms from Nominatim https://operations.osmfoundation.org/policies/nominatim/ ! So reverse search is only accceptable for a small amount of requests!

## Exiftool Command Lines
You need to install exiftool and set path variables accordingly to be able to execute it in target directory. Find some examples her, for more info check out the following sources:

* **[EXIFTOOL FAQ](https://exiftool.org/faq.html 'EXIFTOOL FAQ')**
* **[EXIFTOOL EXAMPLES](https://exiftool.org/examples.html 'EXIFTOOL EXAMPLES')**
* **[EXIFTOOL DOCUMENTATION](https://exiftool.org/exiftool_pod.html 'EXIFTOOL DOCUMENTATION')**
* **[EXIFTOOL GEOTAGGING](https://exiftool.org/geotag.html 'EXIFTOOL GEOTAGGING')**

In [ ]:
# here's import of all packages required to execute below examples
import os
from importlib import reload
from datetime import datetime
from datetime import timedelta
import pytz

import image_meta
import image_meta.persistence
import image_meta.util
import image_meta.geo
reload(image_meta)
reload(image_meta.persistence)
reload(image_meta.util)
reload(image_meta.geo)

# Import classes
from image_meta.persistence import Persistence as P
from image_meta.util import Util as U
from image_meta.geo import Geo as G
from image_meta.exif import ExifTool as E

# Sample Data
coords = {"Stuttgart":{"lat":48.7835,"lon":9.1850},
          "Tübingen":{"lat":48.52027,"lon":9.05361}}
lat,lon = list(coords["Tübingen"].values())
# OSM Link can be constructed like
print(f"Tübingen OSM Link -> https://www.openstreetmap.org/#map=15/{lat}/{lon}")
print("Reverse Search link:")
# Reverse Search url for this link is (click to see the data)
print(f"""https://nominatim.openstreetmap.org/reverse?format=jsonv2&lat={lat}\
&lon={lon}&addressdetails=16&namedetails=1&extratags=1""")
# timezone
tz_local = pytz.timezone("Europe/Berlin")
tz_utc = pytz.timezone("UTC")

# Reading configuration data (modify config.json to your own environment)
curr_path = os.path.abspath(os.getcwd())
config_path = os.path.join(curr_path,"config.json")
print(f"Reading config File: {config_path}")
config = P.read_json(config_path)
print("Configuration Data:",config)
exiftool_path = config["exiftool"]

In [ ]:
## Exiftool Command Line Examples

In [ ]:
import os

curr_path = os.path.abspath(os.getcwd())
print(f"Current Path {curr_path}")
#get test.jpg samples in samples subdirectory
sample_file = os.path.join(curr_path, "samples","img_test.jpg")
sample_json = os.path.join(curr_path, "samples","img_test.json")
if os.path.isfile(sample_file):
    # exiftool needs to be installed and available at command line in work dir
    print("--- Output of all EXIF subsegment metadata Containing date information ---")
    !exiftool -G -s -exif:*date* {sample_file}
    print("--- Same Data as json ---") 
    !exiftool -G -s -j -exif:*date* {sample_file}
    print("--- Same Data as json / Short Version without groups ---") 
    !exiftool -s -j -exif:*date* {sample_file}        
    print("--- Output into json file (in samples folder ) ---")
    !exiftool -G -s -j -exif:*date* {sample_file} > {sample_json}
    #now we can read the json into dict
    print("reading from json into dict:")
    metadata_dict = P.read_json(sample_json)
    print(metadata_dict)

# Persistence Module
Operations for saving / loading / copying data in various formats (txt, json, gpx xml format)

### Copy Files

In [ ]:
# copy files from one directory to another (optional filtered by file extension)
curr_path = os.path.abspath(os.getcwd())
src_path = os.path.abspath("./samples")
trg_path = os.path.abspath("./work")
ext = ""
copied_files = P.copy_files(src_path=src_path,trg_path=trg_path,ext="")
print("--- Copying Files ---")
copied_files

### Read GPX files
Read gpx xml files, also support heart rate and cadence from fitness watch. Key is UTC timestamp.

In [ ]:
# reading gpx data from work directory
work_path = os.path.abspath("./work")
gpx_path = os.path.join(work_path,"sample_gpx.gpx")
gpx = P.read_gpx(gpx_path)
# print the first 3 gps points
gpx_keys = list(gpx.keys())[:3] # timestamps used as keys
[(k,datetime.utcfromtimestamp(k).strftime("%m/%d/%Y, %H:%M:%S")
  ,gpx[k]) for k in gpx_keys]

### Get File Paths
The `Persistence.get_file_list(path,file_type_filter=None)` method allows you to put in a single file ref, a list of files, or a path or a list of paths or a combinaton to get the full paths of files in a list. the file type filter allows you to filter for files with only specified extensions

In [ ]:
work_path = os.path.abspath("./work")
print("--- all files in work path ---") 
print(P.get_file_list(work_path)) 
print("\n--- only gpx ---") 
print(P.get_file_list(work_path,file_type_filter="gpx"))
print("\n--- only certain files ---")
f = [r"C:\30_Entwicklung\WORK_JUPYTER\root\image_meta\work\gps.args",
     r"C:\30_Entwicklung\WORK_JUPYTER\root\image_meta\work\IMG_20200615_114528792_NO_META.jpg"]
print(P.get_file_list(f))

# Geo Module

In [ ]:
# convert lat lon into cartesian (X,Y,Z) coordinates
c1 = list(coords["Tübingen"].values())
G.latlon2cartesian(c1)

In [ ]:
# calculate the distance in km of two coordinates (to initialize data, run the first cell above)
c1 = list(coords["Tübingen"].values())
c2 = list(coords["Stuttgart"].values())
G.get_distance(c1,c2,debug=True)

# Utils Module

### Datetime Conversion
get_timestamp returns timestamp, assumption time string is given in UTC (needs to be converted into UTC before)

In [ ]:
from pytz import timezone
tz = timezone('Europe/Berlin')
utc = timezone('UTC')
# get UTC Timestamp from Date String conforming to format ####-##-##T##:##:##Z / (+/-)##:##  
now = datetime.now().astimezone(utc)
print("Now:",now)
#now = datetime(2020, 1, 17,20,10,12)
now_s = now.strftime("%Y-%m-%dT%H:%M:%SZ")
now_ts = U.get_timestamp(now_s)
print(f"Now DateTime {now} -> Now String: {now_s} -> UTC Timestamp {now_ts}")
#convert back from timestamp
utc_dt = tz_utc.localize(datetime.utcfromtimestamp(now_ts))
cet_dt = utc_dt.astimezone(tz_local)
print("Timestamp -> Datetime UTC",utc_dt," -> Datetime Local",cet_dt)
print("UTC Offset",cet_dt.utcoffset()," Timezone",cet_dt.tzinfo,
      " Daylight Saving Time OFFSET",cet_dt.tzinfo.dst(cet_dt))

In [ ]:
# More examples > all same dates but differently formatted / default time Europe / Berlin 
dates = ["2020-05:12 13:23:12","2020-05-12T11:23:12Z",
         "2020-05-12T11:23:12.000Z","2020-05-12T13:23:12+02:00"]
for date_s in dates:
    print(U.get_timestamp(date_s,debug=True))

### Timestamp Offset
Calculate offset when GPS time is differing from Camera time

In [ ]:
# Different time formats as string allowed see above
s_gps = "2020-05-12T13:23:20+02:00" 
s_cam = "2020-05:12 13:23:12"

offset = U.get_time_offset(time_camera=s_cam,time_gps=s_gps,debug=True) // timedelta(seconds=1)
print(f"Offset Camera - GPS is {offset} seconds")

### Binary Approximate Search
Find the "floor" element in a sorted list of numbers that comes close to passed value 

In [ ]:
sorted_list = sorted([5,2.2,3.5,2,6,9,12])
print(sorted_list)
value1 = 5
idx1 = U.get_nearby_index(value1,sorted_list)
print("value",value1,"index ",idx1," list value ->",sorted_list[idx1])
value1 = 4
idx1 = U.get_nearby_index(value1,sorted_list)
print("value",value1,"index ",idx1," list value ->",sorted_list[idx1])
value1 = 0
idx1 = U.get_nearby_index(value1,sorted_list)
print("value",value1,"index ",idx1," list value ->",sorted_list[idx1])
value1 = 13
idx1 = U.get_nearby_index(value1,sorted_list)
print("value",value1,"index ",idx1," list value ->",sorted_list[idx1])

In [ ]:
# here you can see how it chunks the sorted list into halfs
value1 = 11.5
idx1 = U.get_nearby_index(value1,sorted_list,debug=True)
print("value",value1,"index ",idx1," list value ->",sorted_list[idx1])

# Exif Module

### Metadata Hierarchy
In photo management programs you often can maintain tags as hierarchies and export them as text file. In this file, a hierarchy level is represented as tab character. From this, you can construct hierarchical meta tags (stored as XMP:HierarchicalSubject in image metadata). The following method will read a hierarchy metadata file and put them into a dict with the "leaf" tag as dict key. This way, you can maintain a hierarchy and automaticall get the hierachical meta tag by just maintaining the hierarchy in a text file.   

In [ ]:
# read the meta file (needs to be UTF8)
curr_path = os.path.abspath(os.getcwd())
#get test hierarchy samples in samples subdirectory
sample_hier = os.path.join(curr_path, "work","test_hier.txt")
if not os.path.isfile(sample_hier):
    raise Exception(f"{sample_hier} NOT FOUND")
    
lines = P.read_file(sample_hier)
print("-----------HIERARCHY-------------")
for line in lines:
    print(line.strip('\n'))
print("-----------OUTPUT-------------")
h_tag_dict = E.create_metahierarchy_from_str(lines,debug=False)
print(h_tag_dict)
tag = "Tübingen"
print("-----------Example-------------")
print(f"Tag <{tag}> has hierarchical attribute <{h_tag_dict[tag]}>")

### Process Images with Exiftool
In Class `ExifTool` executable will be triggered by `execute` method receiving control parameters and file list. In the constructor the image folder and the path to the Exiftool executable needs to be supplied. 
Convenience wrapper methods for handling metadata are supplied and described here. 

In [ ]:
curr_path = os.path.abspath(os.getcwd())
sample_jpg = os.path.join(curr_path, "work","img_test.jpg")
exif_tool_loc = exiftool_path # define location in file config.json  
print("Exiftool: ",exif_tool_loc)

if not os.path.isfile(exif_tool_loc):
    raise Exception(f"EXIFTOOL NOT FOUND at location {exif_tool_loc}")

if not os.path.isfile(sample_jpg):
    raise Exception(f"file {sample_jpg} NOT FOUND")

# # important: needs to be handled via "with" command (-> executing "__enter__" method)    
with E(exif_tool_loc) as exiftool:
    # collects data of several files in one dictionary
    try:
        meta_dict = exiftool.get_metadict_from_img(sample_jpg)
    except:
        print(f"error reading file {sample_jpg} check if it is there")

file_list = meta_dict.keys()

for jpg_file in file_list:
    print(f"--- File {jpg_file} ---")    
    meta_list = meta_dict[jpg_file]
    
    for meta in meta_list:
        print(f"[{meta}] ->  {meta_list[meta]}")            

In [ ]:
# exif example: calculate time offset with Utility Module
s_gps = "2020:01:03 13:51:43" # time read from image
# reading meta data, from example above
s_cam = meta_list['CreateDate']
offset = U.get_time_offset(time_camera=s_cam,time_gps=s_gps,debug=False) // timedelta(minutes=1)
print(f"GPS time:{s_gps} Cam Time:{s_cam}, Offset Camera - GPS is {offset} minutes")